# Use ConceptNet for improving word tokenizer

In [1]:
from importlib import resources
import pandas as pd
with resources.path('word_embeddings', 'mini.zh.h5') as hdf_file:
    embeddings: pd.DataFrame = pd.read_hdf(hdf_file)

In [2]:
all_vocabs = embeddings.index.str.replace('/c/zh/', '')
all_vocabs

Index(['##cm', '0', '0.5', '1', '1s', '2', '3', '3c', '3p', '3q',
       ...
       '龟毛', '龟甲', '龟缩', '龟背竹', '龟船', '龟裂', '龠', '龢', '𫓧', '𫚉鱼'],
      dtype='object', length=270235)

In [3]:
from ezra.resources import bible as unv

In [4]:
from tqdm import tqdm
tqdm.pandas()

In [5]:
#occurrence = all_vocabs.to_series().progress_apply(lambda vocab: unv.text.str.count(vocab).sum())
#occurrence[occurrence > 0].to_csv('ccn_occurrence.csv')

In [11]:
occurrence = pd.read_csv('../data/ccn_occurrence.csv', index_col=0)

In [13]:
from ezra.conceptnet_strategy import ConceptNetStrategy
strategy = ConceptNetStrategy()

In [14]:
import numpy as np
new_vocabs = np.setdiff1d(occurrence.index, strategy._words_vec.index)

In [15]:
len(new_vocabs)

6017

In [160]:
def create_assertions_oov()
    ccn = pd.read_csv('../data/conceptnet-assertions-5.7.0.csv.gz', sep='\t', header=None, chunksize=10000)
    ccn_new = []
    for batch in tqdm(ccn):
        isnew = batch[2].str.replace('/c/zh/', '').isin(new_vocabs) | batch[3].str.replace('/c/zh/', '').isin(new_vocabs)
        if isnew.any():
            ccn_new.append(batch[isnew])
    ccn_new_df = pd.concat(ccn_new)
    ccn_new_df.to_csv('../data/conceptnet-assertions-5.7.0-oov.csv', sep='\t', index=None)

#create_assertions_oov()

3408it [07:30,  7.57it/s]


In [ ]:
ccn_new_df = pd.read_csv('../data/conceptnet-assertions-5.7.0-oov.csv', sep='\t')

In [175]:
import json

def group_contributor(contributor):
    if contributor.startswith('/s/contributor/petgame/'):
        return 'petgame'
    else:
        return contributor

def extract_source(info):
    return [group_contributor(src['contributor']) for src in json.loads(info)['sources']]

ccn_new_df['source'] = ccn_new_df[4].transform(extract_source)

In [189]:
new_vocab_source = pd.concat([
    pd.concat([ccn_new_df[2], ccn_new_df[3]]),
    pd.concat([ccn_new_df['source'], ccn_new_df['source']])
], axis=1)
new_vocab_source

,0,source
65819,/c/zh/凡,[/s/resource/wiktionary/en]
65865,/c/zh/晚上/a,[/s/resource/wiktionary/fr]
65900,/c/zh/陰/n,[/s/resource/wiktionary/fr]
154726,/c/zh/xo_醬,[petgame]
154737,/c/zh/一些人,[petgame]
...,...,...
34062533,/c/zh/過年,[petgame]
34062540,/c/zh/生孩子,[petgame]
34062552,/c/zh/睡不著,[petgame]
34062599,/c/zh/找食物,"[petgame, petgame, petgame]"


In [201]:
new_vocab_source.index = new_vocab_source[0].str.replace('/c/zh/', '')
new_vocab_source = new_vocab_source[new_vocab_source.index.isin(new_vocabs)][['source']]
new_vocab_source

,source
0,
一些人,[petgame]
一塊錢,[petgame]
一根頭髮,[petgame]
一百塊,[petgame]
一群人,[petgame]
...,...
過年,[petgame]
生孩子,[petgame]
睡不著,[petgame]


In [225]:
grp = new_vocab_source.groupby(0).agg(lambda g: g.to_list())
grp['source'] = grp.source.transform(lambda nl: np.unique([item for l in nl for item in l]))
grp

,source
0,
0,"[/s/resource/dbpedia/2015/en, /s/resource/wikt..."
1,"[/s/resource/dbpedia/2015/en, /s/resource/unic..."
2,"[/s/resource/dbpedia/2015/en, /s/resource/unic..."
3,"[/s/resource/dbpedia/2015/en, /s/resource/unic..."
4,[/s/resource/unicode/cldr/32.0.1]
...,...
鼻孔內,[petgame]
鼻孔裡,[petgame]
齊齊,[/s/resource/wiktionary/en]


In [247]:
new_vocabs_by_sources = grp.explode('source').groupby('source').apply(lambda x: x.index.to_list())

In [255]:
new_vocabs_by_sources

source
/s/resource/cc_cedict/2017-10      [a, 一個人, 一個樣, 一句話, 一字, 一定要, 一意, 一百一, 一空, 一舉, 一...
/s/resource/dbpedia/2015/en        [0, 1, 2, 3, 5, 9, a, 一杯酒, 七七節, 七十士譯本, 不等, 中斷,...
/s/resource/unicode/cldr/32.0.1    [1, 2, 3, 4, 5, 9, a, 一杯酒, 上衣, 不懂, 不滿意, 不相信, 不...
/s/resource/wiktionary/de          [一千一, 一行, 一起, 丁, 三十一, 不同的, 久長, 二百五, 云, 亦, 人世, ...
/s/resource/wiktionary/en          [0, 1, 一上, 一丟, 一人, 一個人, 一國, 一團, 一夜之間, 一夥, 一意, ...
/s/resource/wiktionary/fr          [一起, 丁, 下一個, 不朽, 中原, 中止, 于, 云, 亦, 人參, 人格, 人體, ...
petgame                            [一中, 一些人, 一個人, 一個人去, 一個兒子, 一個名字, 一個家, 一個島, 一個男...
dtype: object

In [262]:
' '.join(new_vocabs_by_sources.iloc[4])

'0 1 一上 一丟 一人 一個人 一國 一團 一夜之間 一夥 一意 一望 一條線 一歇 一發 一百一 一百歲 一空 一節 一舉 一處 一行 一見 一起 一門 一雙 一點也不 丁當 七十三 七十六 七國 三代 三個 三分 三十一 三十九 三十八 三小 三層 三更 三條 三番 上人 上傳 上像 上光 上分 上半 上半截 上合 上回 上坐 上奏 上套 上帳 上座 上德 上愚 上房 上操 上日 上晝 上朝 上正 上水 上油 上活 上無 上照 上燈 上畫 上相 上知 上禮拜 上稅 上第 上算 上糞 上聖 上肢 上腦 上臉 上落 上衣 上表 上裝 上言 上進 上達 上都 上野 上閂 上鞋 上首 上香 上體 上高 下一個 下作 下半 下回 下埃及 下女 下子 下心 下沉 下詔 下達 下門 下頭 不一 不一定 不乾 不亂 不休 不來 不依 不信神 不信神的 不停 不備 不允 不全 不公平 不利 不勞 不厭 不可以 不可信 不可用 不可缺 不吉 不吝 不和 不堪 不外 不多時 不孝 不容易 不實 不寬容 不對 不屈 不差 不干 不從 不復 不忘 不忠 不快 不思 不恨 不情願 不慣 不懂 不懈 不揀 不敗 不敬 不敵 不早 不有 不服從 不朽 不棄 不樂 不比 不治 不注意 不清 不為 不甘 不甚 不生育 不生育的 不知道 不禁 不移 不等 不興 不舉 不解 不計 不識 不足的 不辭 不速 不遵 不關 不離 不願 不體貼 世主 世好 世子 世孫 並不 並起 中上 中二 中亞 中位 中出 中原 中可 中和 中哥 中天 中常 中斷 中日 中止 中正 中游 中用 中的 中看 中聽 中胡 中興 中舉 中藏 中計 中跑 中都 中非 中點 串 丹 主上 主婦 主將 主打 主教 主管 主謀 久長 乏困 乖 九十二 九十五 九十八 也不 也就是 也是 乳哺 亂主 了得 了無 了當 了離 事急 事端 二三 二人轉 二十七 二回 二天 二更 二百五 云 五十一 五十三 五十二 些少 亞基 亞省 亞達月 亦 亨 亭 亮星 人世 人中 人偶 人像 人參 人和 人堆 人從 人性 人情 人格 人權 人煙 人相 人聲 人肉 人臉 人設 人證 人選 人骨 人體 仁 今人 介 仔 他人 他心 他日 他用 仗勢 付 付與 仙 代代 代為 以免 以前的 以往 以是 以祿月 以色列人 以西 以西結 以賽亞書 仰 仰觀 任

In [265]:
'約拿' in strategy._words_vec.index

False